# Demonstration of DataFrame "Nearby" Missing Value Imputer

For some variables, LiveHVE implements a missing value imputation strategy of using the _median_ value variables for properties that are in the same 'census tract'.  This notebook demonstrates how a custom `sklearn` transformer can be used to implement this strategy.

**Note: This notebook requires `sklearn` 1.0 or higher**

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_regression
from sklearn.base import BaseEstimator, TransformerMixin
import sklearn

print(f"sklearn version {sklearn.__version__}")

sklearn version 1.0.2


## Create synthetic data for demonstration

In [2]:
N_SAMPLES = 10
N_FEATURES = 3

In [3]:
X, _ = make_regression(n_samples=N_SAMPLES, n_features=N_FEATURES, random_state=123)

# create some missing values
np.random.seed(123)
idx = np.random.choice(range(N_SAMPLES), size=int(0.5 * N_SAMPLES))
X[idx, 0] = np.nan
idx = np.random.choice(range(N_SAMPLES), size=int(0.3 * N_SAMPLES))
X[idx, 1] = np.nan
idx = np.random.choice(range(N_SAMPLES), size=int(0.5 * N_SAMPLES))
X[idx, 2] = np.nan

ctract = np.random.choice(['123456', '987654'], size=N_SAMPLES, replace=True).reshape(-1, 1)

df = pd.DataFrame(np.concatenate([X, ctract], axis=1), columns=['X00', 'X01', 'X02', 'ctract'])
df = df.astype({'X00': np.float32, 'X01': np.float32, 'X02': np.float32})
df

,X00,X01,X02,ctract
0,1.651437,-0.578600,NaN,987654
1,NaN,NaN,NaN,987654
2,NaN,-0.637752,-1.253881,987654
3,NaN,0.997345,-1.085631,123456
4,1.265936,-0.428913,-2.426679,987654
5,-0.094709,-0.678886,-0.866740,123456
6,NaN,NaN,-0.434351,123456
7,0.737369,0.386186,1.004054,123456
8,-0.861755,-0.140069,-1.428681,123456
9,-0.443982,NaN,NaN,987654


In [4]:
df.dtypes

X00       float32
X01       float32
X02       float32
ctract     object
dtype: object

### Display median values by `ctract` for each data variable

In [5]:
nearby_imputed_values = df.groupby(['ctract']).median()
nearby_imputed_values

,X00,X01,X02
ctract,,,
123456,-0.094709,0.123059,-0.86674
987654,1.265936,-0.578600,-1.84028


In [6]:
overall_imputed_values =df.drop(['ctract'], axis=1).median()
overall_imputed_values

X00    0.321330
X01   -0.428913
X02   -1.085631
dtype: float32

## Custom Transformer for Performing 'Nearby' Missing Value Imputation

There is no standard `sklearn` missing value imputation class that implements a 'Nearby' approach, a custom was built from scratch to implement the LiveHVE requirement.  For purposes of this notebook, the terms 'groupby', 'census tract' or 'ctract' are interchangeable.  

It is assumed that all census tracts are from the same county.  If census tract data from mulitple counties are mixed, then inconsistent results may occur.

In addition to the using the summary statistic, such as'median' value, it is possible to specify the use of 'mean' and a 'constant' value.

Lastly, to handle the case where a 'census tract' is not seen during the `fit()` method, the custom class will use the overall summary statistic for the particular column.

In [7]:
class DFNearbyImputer(BaseEstimator, TransformerMixin):
    def __init__(self, strategy:str='median', fill_value:float=None, add_indicator:bool=False,
                 groupby:str=None, keep_groupby:bool=False) -> None:
        self.strategy = strategy
        self.fill_value = fill_value
        self.add_indicator = add_indicator
        self.groupby = groupby
        self.keep_groupby = keep_groupby
        self.feature_names_in_ = None
        self.nearby_imputed_values_ = None
        self.overall_imputed_values_ = None

        self.summary_stats = {
            'median': pd.DataFrame.median,
            'mean': pd.DataFrame.mean,
            'constant': self._fill_in_constant
        }

    def _fill_in_constant(self, X, **kwargs):
        if isinstance(X, pd.DataFrame):
            # drop the groupby column
            # return series of fill value for each column
            column_names = X.drop([self.groupby], axis=1).columns
            values = np.ones([len(column_names)]) * self.fill_value
            return pd.Series({k:v for k, v in zip(column_names, values)})
        elif isinstance(X, pd.Series):
            return self.fill_value
        else:
            raise ValueError(
                f"Expected pandas DataFrame or Series, instead found {type(X)}"
            )

    def fit(self, X:pd.DataFrame, y=None) -> object:
        # collect column names except for groupby column
        self.feature_names_in_ = X.drop([self.groupby], axis=1).columns.to_list()

        # compute missing value imputation for each groupby value for all columns
        self.nearby_imputed_values_ = X.groupby([self.groupby]).apply(self.summary_stats[self.strategy],
                                                               numeric_only=True)

        # compute default missing value imputation for the case that groupby value was not seen
        # during fit() method
        self.overall_imputed_values_ = X.drop([self.groupby], axis=1).apply(self.summary_stats[self.strategy])

        return self

    def transform(self, X:pd.DataFrame) -> pd.DataFrame:
        # create shallow copy to avoid altering original object
        df = X.copy()
        for col in df.drop([self.groupby], axis=1).columns:
            for group in X[self.groupby].unique():
                # for each column and for each groupby value find missing
                # value locations to fill
                mask = (X[col].isna()) & (X[self.groupby] == group)
                try:
                    # get summary statistic for "nearby" observations
                    # where() method requires logical not of mask values
                    df.loc[:, col] = df.loc[:, col].where(np.logical_not(mask), other=self.nearby_imputed_values_.loc[group, col])
                except KeyError:
                    # if no summary statistic exists for the "neraby" location, use overall default stats
                    # where() method requires logical not of mask values
                    df.loc[:, col] = df.loc[:, col].where(np.logical_not(mask), other=self.overall_imputed_values_.loc[col])

        if self.add_indicator:
            # add indicator variables if requested
            missing_indicator = X.drop([self.groupby], axis=1).isna()
            df = pd.DataFrame(
                np.concatenate([df, np.int8(missing_indicator)], axis=1),
                columns=[
                    df.columns.to_list() + [f"j{c}" for c in df.drop([self.groupby], axis=1).columns]
                ]
            )

        if self.keep_groupby:
            return df
        else:
            return df.drop([self.groupby], axis=1)


### Use nearby values

In [8]:
impute1 = DFNearbyImputer(groupby='ctract')
impute1.fit_transform(df)

,X00,X01,X02
0,1.651437,-0.578600,-1.840280
1,1.265936,-0.578600,-1.840280
2,1.265936,-0.637752,-1.253881
3,-0.094709,0.997345,-1.085631
4,1.265936,-0.428913,-2.426679
5,-0.094709,-0.678886,-0.866740
6,-0.094709,0.123059,-0.434351
7,0.737369,0.386186,1.004054
8,-0.861755,-0.140069,-1.428681
9,-0.443982,-0.578600,-1.840280


### Check if previously computed values match fitted values

In [9]:
# check census tract values
impute1.nearby_imputed_values_ == nearby_imputed_values

,X00,X01,X02
ctract,,,
123456,True,True,True
987654,True,True,True


In [10]:
# check overall 
impute1.overall_imputed_values_ == overall_imputed_values

X00    True
X01    True
X02    True
dtype: bool

### Use nearby values and keep census tract column

In [11]:
impute2 = DFNearbyImputer(groupby='ctract', keep_groupby=True)
impute2.fit_transform(df)

,X00,X01,X02,ctract
0,1.651437,-0.578600,-1.840280,987654
1,1.265936,-0.578600,-1.840280,987654
2,1.265936,-0.637752,-1.253881,987654
3,-0.094709,0.997345,-1.085631,123456
4,1.265936,-0.428913,-2.426679,987654
5,-0.094709,-0.678886,-0.866740,123456
6,-0.094709,0.123059,-0.434351,123456
7,0.737369,0.386186,1.004054,123456
8,-0.861755,-0.140069,-1.428681,123456
9,-0.443982,-0.578600,-1.840280,987654


### Use nearby values with missing value indicator variables

In [12]:
impute3 = DFNearbyImputer(groupby='ctract', keep_groupby=True, add_indicator=True)
impute3.fit_transform(df)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,-1.84028,987654,0,0,1
1,1.265936,-0.5786,-1.84028,987654,1,1,1
2,1.265936,-0.637752,-1.253881,987654,1,0,0
3,-0.094709,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,-0.094709,0.123059,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,-0.5786,-1.84028,987654,0,1,1


### Use nearby values with indicator variables and remove census tract column

In [13]:
impute4 = DFNearbyImputer(groupby='ctract', keep_groupby=False, add_indicator=True)
impute4.fit_transform(df)

,X00,X01,X02,jX00,jX01,jX02
0,1.651437,-0.5786,-1.84028,0,0,1
1,1.265936,-0.5786,-1.84028,1,1,1
2,1.265936,-0.637752,-1.253881,1,0,0
3,-0.094709,0.997345,-1.085631,1,0,0
4,1.265936,-0.428913,-2.426679,0,0,0
5,-0.094709,-0.678886,-0.86674,0,0,0
6,-0.094709,0.123059,-0.434351,1,1,0
7,0.737369,0.386186,1.004054,0,0,0
8,-0.861755,-0.140069,-1.428681,0,0,0
9,-0.443982,-0.5786,-1.84028,0,1,1


### Create unseen census tract data

In [14]:
df2 = pd.concat(
    [
        df,
        pd.DataFrame([{'X00': np.nan, 'X01': np.nan, 'X02': np.nan, 'ctract':'111222'}])
    ],
    ignore_index=True
)
df2

,X00,X01,X02,ctract
0,1.651437,-0.578600,NaN,987654
1,NaN,NaN,NaN,987654
2,NaN,-0.637752,-1.253881,987654
3,NaN,0.997345,-1.085631,123456
4,1.265936,-0.428913,-2.426679,987654
5,-0.094709,-0.678886,-0.866740,123456
6,NaN,NaN,-0.434351,123456
7,0.737369,0.386186,1.004054,123456
8,-0.861755,-0.140069,-1.428681,123456
9,-0.443982,NaN,NaN,987654


In [15]:
# use previously fitted imputer that did not see the added census tract
impute3.transform(df2)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,-1.84028,987654,0,0,1
1,1.265936,-0.5786,-1.84028,987654,1,1,1
2,1.265936,-0.637752,-1.253881,987654,1,0,0
3,-0.094709,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,-0.094709,0.123059,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,-0.5786,-1.84028,987654,0,1,1


In [16]:
# display overall imputed values used when unseen census tract value is encountered
impute3.overall_imputed_values_

X00    0.321330
X01   -0.428913
X02   -1.085631
dtype: float64

### Use 'mean' for summary statistic

In [17]:
impute5 = DFNearbyImputer(strategy='mean', groupby='ctract', keep_groupby=True, add_indicator=True)
impute5.fit_transform(df)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,-1.84028,987654,0,0,1
1,0.824464,-0.548421,-1.84028,987654,1,1,1
2,0.824464,-0.637752,-1.253881,987654,1,0,0
3,-0.073032,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,-0.073032,0.141144,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,-0.548421,-1.84028,987654,0,1,1


In [18]:
impute5.nearby_imputed_values_

,X00,X01,X02
ctract,,,
123456,-0.073032,0.141144,-0.56227
987654,0.824464,-0.548421,-1.84028


In [19]:
impute5.overall_imputed_values_

X00    0.375716
X01   -0.154384
X02   -0.927416
dtype: float64

In [20]:
impute5.transform(df2)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,-1.84028,987654,0,0,1
1,0.824464,-0.548421,-1.84028,987654,1,1,1
2,0.824464,-0.637752,-1.253881,987654,1,0,0
3,-0.073032,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,-0.073032,0.141144,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,-0.548421,-1.84028,987654,0,1,1


### Use 'constant' for imputed value

In [21]:
impute6 = DFNearbyImputer(strategy='constant', fill_value=0.0, 
                          groupby='ctract', keep_groupby=True, add_indicator=True)
impute6.fit_transform(df)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,0.0,987654,0,0,1
1,0.0,0.0,0.0,987654,1,1,1
2,0.0,-0.637752,-1.253881,987654,1,0,0
3,0.0,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,0.0,0.0,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,0.0,0.0,987654,0,1,1


In [22]:
impute6.transform(df2)

,X00,X01,X02,ctract,jX00,jX01,jX02
0,1.651437,-0.5786,0.0,987654,0,0,1
1,0.0,0.0,0.0,987654,1,1,1
2,0.0,-0.637752,-1.253881,987654,1,0,0
3,0.0,0.997345,-1.085631,123456,1,0,0
4,1.265936,-0.428913,-2.426679,987654,0,0,0
5,-0.094709,-0.678886,-0.86674,123456,0,0,0
6,0.0,0.0,-0.434351,123456,1,1,0
7,0.737369,0.386186,1.004054,123456,0,0,0
8,-0.861755,-0.140069,-1.428681,123456,0,0,0
9,-0.443982,0.0,0.0,987654,0,1,1
